# Work on Compensatory Indels
**Mark Richard, with Yaniv Brandvain**

The current state of the work has been to extend methods used on 4 individuals to a much larger data-set involving 180 individuals. This large data set is based on a [2013 paper](http://www.nature.com/ng/journal/v45/n8/full/ng.2678.html?foxtrotcallback=true) by the Nordborg research group (first author Long). The GitHub site that contains the data is [here](https://github.com/Gregor-Mendel-Institute/swedish-genomes).

This document outlines the code and procceses used to recreate the results from the Long 2013 paper, and hopefully extend them.

*Note: The code that follows is very dependent on the authors file system. This is as much a note to the author to change the structure of the code to make it more readily useable for anyone.*

## Creating a useable annotation file

On the GitHub site, the annotation file in the /Indel/Original folder was downloaded. The choice was made to not use the imputed data, as it was not consistently available across all datasets.

The annotation file consists of sets of 2 lines: one line defining the beginning of an indel, and one line defining the end of the same indel. Many of these indels go across genes; we are only interested in looking at indels that lie entirely inside a single gene, and so the following code was used to create a new annotation file (adjustedAnnotation) that only contained indels that lie entirely inside a single gene.

In [ ]:
file = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Genomes/samtools.2.1.s.filled.e.noimpute.annotation')
newFile = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/adjustedAnnotation.txt','w')
fileList = []
for i in file:
    if ';' not in i.strip('\n').split('\t')[7]:
        fileList.append(i.strip('\n').split('\t'))

for i in fileList:
    for j in i:
        newFile.write(j)
        newFile.write('\t')
    newFile.write('\n')

file.close()
newFile.close()

The code makes use of the fact that the original annotation file separates intergenic information by semicolons: if an indel lies in more than one gene, there will be a semicolon in that column. So, we want all indels that do *not* have a semicolon in the relevant column.

## Combining the Annotation and Info files

The next step was to adjust the original .info file (this contains a list of each indel, one per line, detailing its location, length, the specific event, and whether it was an insertion or deletion). We only took the indels that were left in the adjusted annotation file (i.e. those that lie entirely in a single gene), and created a new file, 180StrainsRead.

In [ ]:
dataSource = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Genomes/samtools.2.1.s.filled.e.noimpute.info')
annot= open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/adjustedAnnotation.txt')
newDataFile = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/180StrainsRead.txt','w')

#sort into list, keeping only chrom, loc, size, bases

dataList = []
for i in dataSource:
    dataList.append(i.strip('\n').split('\t'))
del dataList[0]
del dataList[0]

anList = []
for i in annot:
    anList.append(i.strip('\n').split('\t'))
    
anDict = {}
for i in anList:
    anDict[(i[0],i[1])] = i[7]

for i in dataList:
    newDataFile.write(i[0])
    newDataFile.write('\t')
    newDataFile.write(str(i[1]))
    newDataFile.write('\t')
    if str(i[3][0]) == '-':
        newDataFile.write('-' + str(i[2]))
        newDataFile.write('\t')
    else:
        newDataFile.write(str(i[2]))
        newDataFile.write('\t')
    newDataFile.write(str(i[3]))
    newDataFile.write(str('\t'))
    if (i[0],i[1]) in anDict:
        newDataFile.write(str(anDict[(i[0],i[1])]))
    newDataFile.write('\n')
    

dataSource.close()
newDataFile.close()

The code is doing nothing particularly interesting. The only important thing to note is that the adjustedAnnotation file (`annot` in the code) is split into *pairs* of lines, and the first line in each pair corresponds to some line in the .info original file (`dataSource` in the code). The lines in `dataSource` that happen to be found in `annot` are just put into the 180StrainsRead file (`newDataFile` in the code), with some columns being truncated, and the added bonus of the length of the indel being denoted as positive or negative (for an insertion vs. deletion).

## Add Lengths to individual data

The next step was to take the original .csv file, which contains the information on whether each of the 180 individuals have a particular indel, and match it up with the adjusted annotation files, as well as include the length of the indel. Similar to the previous bit of code, what follows just checks if a line in the .csv file (`csv`) is in the 180StrainsRead file (`reads`), and then just copies the line into the new IndividualIndels file (`newDataFile`), while inserting a column that includes the length of the indel in question.

In [ ]:
csv = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Genomes/samtools.2.1.s.filled.e.noimpute.csv')
reads = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/180StrainsRead.txt')

newDataFile = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/IndividualIndels.txt','w')

##sort csv, reads into lists, create new list

csvList = []
for i in csv:
    csvList.append(i.strip('\n').split(','))

readsList = []
for i in reads:
    readsList.append(i.strip('\n').split('\t'))

csvList[0].insert(2,'len')
csvList[0].insert(2,'gene')
newDataList = []
for i in range(1,len(csvList)):
    newDataList.append(csvList[i])

## Add the lengths into the data list

c = 0
for i in newDataList:
    i.insert(2, readsList[c][2])
    i.insert(2,readsList[c][4])
    c += 1

newDataList.insert(0,csvList[0])
print(newDataList[0])

## Write into file

for line in newDataList:
    for item in line:
        newDataFile.write(item)
        newDataFile.write(',')
    newDataFile.write('\n')

csv.close()
reads.close()
newDataFile.close()

## Differentiate between Indels in Exon regions and Intron regions

The next step is the most important one thus far, since all previous steps have just been filtering the data in pretty simple ways. Now, the goal is to separate indels by whether they occur in an exon or an intron. The adjustedAnnotation file has this information, and the difficulty is that many indels have multiple isoforms. To recreate the data, it is necessary to look at the what isoforms they chose.
*Not on campus, so I cannot access the methods section of the online paper.*
It is also necessary to remember that the indels that exist in exon regions must also be in a coding region, otherwise it is not of interest.

For now, the choice was made to exclusively use the first isoform for each indel. The code below creates two files: IndividualIntronsNew, and IndividualExonsNew. This is because IndividiualExons/Introns (without the "New") were created before, but erroneously. They are only kept for the purpose of having a header reference, as shown in the declaration of `getHeader` in the code below.

In [ ]:
annot = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/adjustedAnnotation.txt')
indiv = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/IndividualIndels.txt')
introns = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/IndividualIntronsNew.txt','w')
exons = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/IndividualExonsNew.txt','w')

getHeader = open('/home/mdrichard05/Dropbox/CompensatoryIndels/Data/Processed/IndividualExons.txt')
header = getHeader.readline()
getHeader.close()

anList = []
for i in annot:
    anList.append(i.strip('\n').split('\t')[0:12])
annot.close()

indivList = []
for i in indiv:
    indivList.append(i.strip('\n').split(',')[0:184])
indiv.close()

intronList = []
exonList = []

for i in anList[::2]:
    if i[9][3] == 'i':
        intronList.append([i[0],i[1]])
    else:
        if i[10][3] == 'C':
            exonList.append([i[0],i[1]])

indivIntronList = []
indivExonList = []

## print(intronList[0],intronList[-1])
introns.write(header)
exons.write(header)
for i in range(1,len(indivList)):
    if [indivList[i][0], indivList[i][1]] in intronList:
        for k in range(0, len(indivList[i])-1):
            introns.write(indivList[i][k])
            introns.write(',')
        introns.write(indivList[i][-1])    
        introns.write('\n')
    if [indivList[i][0], indivList[i][1]] in exonList:
        for k in range(0, len(indivList[i])-1):
            exons.write(indivList[i][k])
            exons.write(',')
        exons.write(indivList[i][-1])
        exons.write('\n')
        
introns.close()
exons.close()

Essentially, the code is just skipping every other line of the adjustedAnnotation file (remember, only the first line in each pair is referenced in other files). It then checks if the first isoform starts with an "i" (for intron). If so, it gets put into the intron list, and printed into the IndividualIntronsNew (`introns`) file. If it does not start with an "i", it is an exon. So, we just look at the next column, which tells us if the first isoform is in a UTR region, or a CDS region. If it is in a CDS region (starts with "C"), it is an indel we are interested in, and it gets put into the IndividualExonsNew (`exons`) file.